In [1]:
#load "Paket.fsx"
Paket.Package
  [ "QBitNinja.Client -Version 1.0.3.39" 
    "NBitcoin -Version 4.0.0.48"
  ]   
#load "Paket.Generated.Refs.fsx"


open NBitcoin 
open QBitNinja.Client 
open QBitNinja.Client.Models 
open System 
open System.Collections.Generic 


U - unsigned 
INT - integer
256 - 256 bits in size

In [2]:
open System.Linq

let client:QBitNinjaClient  = new QBitNinjaClient(Network.TestNet)

// Parse transaction id to NBitcoin.uint256 so the client can eat it
let transactionId = uint256.Parse("516c43b617a6fa6dee5f3e9b34d99365b3b89d1381dc9aebcca1e8822c8cb5f4")

// Query the transaction
let transactionResponse:GetTransactionResponse  = client.GetTransaction(transactionId).Result

let transaction:NBitcoin.Transaction  = transactionResponse.Transaction

let receivedCoins:List<ICoin>  = transactionResponse.ReceivedCoins

let receivedCoins' =
    query {
        for coin in receivedCoins do
            select ((coin.Amount:?>Money), 
                     coin.TxOut.ScriptPubKey, 
                     coin.TxOut.ScriptPubKey.GetDestinationAddress(Network.TestNet))
    }
    
receivedCoins' 
    |> Seq.iter (fun x-> 
                        let amount, paymentScript, address = x
                        printfn "-------------"
                        printfn "amount: %A" (amount.ToDecimal(MoneyUnit.BTC))
                        printfn "pub script: %A" paymentScript // It's the ScriptPubKey
                        printfn "destination address: %A" address)


-------------
amount: 0.55M
pub script: OP_DUP OP_HASH160 4fbd5152456bee3dc0ffe25118a6de17126e5c19 OP_EQUALVERIFY OP_CHECKSIG
destination address: mnnaUEKBKJVFjCqVnVPmwefk6CwuVsyRRj
-------------
amount: 30.30840894M
pub script: OP_HASH160 bbdaa6cdc5793ab520c84d9ee0e910ade78759fb OP_EQUAL
destination address: 2NANWHLLQDRA6vUMMbWyUNPxte8gWUMq719


In [3]:
printfn "%A" transaction 

{
  "hash": "516c43b617a6fa6dee5f3e9b34d99365b3b89d1381dc9aebcca1e8822c8cb5f4",
  "ver": 1,
  "vin_sz": 1,
  "vout_sz": 2,
  "lock_time": 0,
  "size": 250,
  "in": [
    {
      "prev_out": {
        "hash": "a8fd6a998d0d9bbf595b1a5d56d704d61cd42e0205e781286dcd723ebf41ed80",
        "n": 1
      },
      "scriptSig": "0014a0fd701a7079bba901b50cca88e2fd9e793ba255",
      "witness": "3045022100c768c68bb16feb17165075a2309ed152f9db4840b69fc3e1d9c69b504dde4b0202205fb1534b40ecaec0bb742a97004dab6ccfce45a22b80c5c4b08d46973804124801 0306ee7e04746dbe0a41b9258d62c6fde85dd7620b4be62a5e5ce768e5122aabb6"
    }
  ],
  "out": [
    {
      "value": "0.55000000",
      "scriptPubKey": "OP_DUP OP_HASH160 4fbd5152456bee3dc0ffe25118a6de17126e5c19 OP_EQUALVERIFY OP_CHECKSIG"
    },
    {
      "value": "30.30840894",
      "scriptPubKey": "OP_HASH160 bbdaa6cdc5793ab520c84d9ee0e910ade78759fb OP_EQUAL"
    }
  ]
}


In [4]:
printfn "amount: %A" transaction.Inputs.Count 

amount: 1


In [5]:
printfn "amount: %A" transaction.Outputs.Count 

amount: 2


In [6]:
let transactionId' =  uint256.Parse("a8fd6a998d0d9bbf595b1a5d56d704d61cd42e0205e781286dcd723ebf41ed80")

let transactionResponse':GetTransactionResponse  = client.GetTransaction(transactionId').Result

let transaction':NBitcoin.Transaction  = transactionResponse'.Transaction

printfn "%A" transaction' 

{
  "hash": "a8fd6a998d0d9bbf595b1a5d56d704d61cd42e0205e781286dcd723ebf41ed80",
  "ver": 1,
  "vin_sz": 1,
  "vout_sz": 2,
  "lock_time": 0,
  "size": 250,
  "in": [
    {
      "prev_out": {
        "hash": "3b8c3a55fcd5a629cd290b4594e8eb6b947c758a2edbe6106102d0c60f51759e",
        "n": 1
      },
      "scriptSig": "001469e90107a63420ec295a2267f365786dd8dc169d",
      "witness": "3045022100a91b5b579f9adc26815e37260a9137d8e86c0cabdfd2baa3116a62e43564374f022004041fa6b5fb5dbdacd07089a180502599427931fb7e66d80302b94b9892660a01 0374931351c5204762b274c54b2c2ed30e02341168a24287fb5f259f19591d8e9f"
    }
  ],
  "out": [
    {
      "value": "0.00214842",
      "scriptPubKey": "OP_DUP OP_HASH160 a053bffe8bfc5f62e83bc9fe3e715638202fd70a OP_EQUALVERIFY OP_CHECKSIG"
    },
    {
      "value": "30.85940894",
      "scriptPubKey": "OP_HASH160 c6a37bfc1ce7449b9b1079220865951358dbe429 OP_EQUAL"
    }
  ]
}


In [9]:
let prevOut = transaction.Inputs.First().PrevOut
printfn "%A" (prevOut)

a8fd6a998d0d9bbf595b1a5d56d704d61cd42e0205e781286dcd723ebf41ed80-1


In [10]:
let transactionId'' =  prevOut.Hash 
let transactionResponse'':GetTransactionResponse  = client.GetTransaction(transactionId'').Result

let transaction'':NBitcoin.Transaction  = transactionResponse''.Transaction

printfn "%A" transaction'' 

{
  "hash": "a8fd6a998d0d9bbf595b1a5d56d704d61cd42e0205e781286dcd723ebf41ed80",
  "ver": 1,
  "vin_sz": 1,
  "vout_sz": 2,
  "lock_time": 0,
  "size": 250,
  "in": [
    {
      "prev_out": {
        "hash": "3b8c3a55fcd5a629cd290b4594e8eb6b947c758a2edbe6106102d0c60f51759e",
        "n": 1
      },
      "scriptSig": "001469e90107a63420ec295a2267f365786dd8dc169d",
      "witness": "3045022100a91b5b579f9adc26815e37260a9137d8e86c0cabdfd2baa3116a62e43564374f022004041fa6b5fb5dbdacd07089a180502599427931fb7e66d80302b94b9892660a01 0374931351c5204762b274c54b2c2ed30e02341168a24287fb5f259f19591d8e9f"
    }
  ],
  "out": [
    {
      "value": "0.00214842",
      "scriptPubKey": "OP_DUP OP_HASH160 a053bffe8bfc5f62e83bc9fe3e715638202fd70a OP_EQUALVERIFY OP_CHECKSIG"
    },
    {
      "value": "30.85940894",
      "scriptPubKey": "OP_HASH160 c6a37bfc1ce7449b9b1079220865951358dbe429 OP_EQUAL"
    }
  ]
}
